In [20]:
import pandas as pd

In [21]:
def read_command(path):
    with open(path) as f:
        lines = f.readlines()

    com = []
    count = 0
    for l in lines:
        if count == 0 and '# Example' in l:
            count += 1
        elif count == 1 and l == '#\n':
            count += 1
        elif count == 2 and '########################' in l:
            count += 1
        elif count ==3 and l == '\n':
            count += 1
        elif count == 4:
            com.append(l.strip())
            count = 0

    return com

def seperate_comma(com_lst):
    raw_lst = [[ele if ele[-1] != ',' else ele[:-1] + ' ' + ',' for ele in command.split()] for command in com_lst]
    com_lst = [' '.join(ele) for ele in raw_lst]
    return com_lst

def read_all(num_files):
    com_lst = []
    for i in range(1,num_files+1):
        path = './Examples/EGPSR Examples_' + str(i)+ '.txt'
        # path = './Examples/EGPSR Examples.txt'
        com_lst.extend(read_command(path))

    com_lst = seperate_comma(com_lst)
    return com_lst

num_files = 1

com_lst = read_all(num_files)
# print(com_lst[3])

print('len: ',len(com_lst))
print(com_lst[:10])

len:  974
['Say -B_greet- hello -I_greet- to the person -i- wearing -i- earrings -i- -i- -B_greet_per- in the guest -i- room -B_greet_dest-', 'Robot please tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- men -i- are -i- leaning -i- against -i- the -i- wall -I_tell_what- in -I_tell_what- the -I_tell_what- garden -I_tell_what-', 'Take -B_guide- the person -B_guide_per- at the shoe -i- rack -B_guide_sour- to the exit -B_guide_dest-', 'Come -B_follow- behind -I_follow- the child -B_follow_per- in the bathroom -B_follow_sour-', 'Could you offer -B_take- something -B_take_obj- to -I_take_obj- drink -I_take_obj- to all -B_take_dest- the -I_take_dest- men -i- wearing -i- sweaters -i- -I_take_dest- in the office -B_take_dest-', 'Could you please give -B_take- me -B_take_dest- some file -B_take_obj- from the oven -B_take_sour-', 'Could you please accompany -B_follow- the person -B_follow_per- at the closet -B_follow_sour-', 'Follow -B_follow- the oldest -i- person -B_follow_p

In [22]:
for i,s in enumerate(com_lst):
   if i == 1000:
       break
   if 'me -B_tell_dest- how -B_tell_what-' in s:
       print(s)

Robot please tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- men -i- are -i- leaning -i- against -i- the -i- wall -I_tell_what- in -I_tell_what- the -I_tell_what- garden -I_tell_what-
Please tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- stationary -I_tell_what- items -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- drying -i- rack -I_tell_what-
Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- gray -I_tell_what- file -I_tell_what- objects -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- vanity -I_tell_what- please
Please tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- men -i- are -i- standing -i- on -i- one -i- foot -I_tell_what- in -I_tell_what- the -I_tell_what- lab -I_tell_what-
Tell -B_tell- me -B_tell_dest- how -B_tell_what- many -I_tell_what- blue -I_tell_what- file -I_tell_what- objects -I_tell_what- are -I_tell_what- in -I_tell_what- the -I_tell_what- shoe -i- rac

In [23]:
def find_sub_list(sub_list,this_list):
    start = this_list.index(sub_list[0])
    return (start,start+len(sub_list))

def process_person_pro(com_lst,path):
    lines = [line.strip() for line in open(path) if line.strip() != '' and line.strip() != 'it']
    pairs_lst = [(lines[i].split(' '),lines[i+1]) for i in range(0,len(lines)//2,2)] #[([B-referee, I-referee],referral),([B-referee, I-referee],referral),....]
    # print(pairs_lst)
    person_pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        lst = com.split()
        cur_referee  = pairs_lst[0][0]
        cur_referral = pairs_lst[0][1]

        words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
        str_referee = ' '.join(cur_referee).strip()
        if str_referee in words_lst and cur_referral in words_lst:
            # ------------ store command idx ------------
            person_pro_dict['idx'].append(i)

            # ------------ store original command  ------------
            person_pro_dict['words'].append(' '.join(words_lst))

            # ------------ store referral, the pronoun in word command ------------
            referral_idx = words_lst.index(cur_referral)
            person_pro_dict['referral'].append((referral_idx,cur_referral))

            # ------------ store referee, the word being referred ------------
            referee_idx = find_sub_list(cur_referee,words_lst)
            #print(referee_idx)
            person_pro_dict['referee'].append((referee_idx,str_referee))

            pairs_lst.pop(0)
            if len(pairs_lst) == 0:
                break
    return person_pro_dict

# path = './Examples/pros_test.txt'
path = './Examples/pros.txt'

person_pro_dict = process_person_pro(com_lst,path)
per_df = pd.DataFrame(person_pro_dict)
per_df

,idx,referee,referral,words
0,8,"((8, 9), Kendall)","(12, her)","Look for Sia in the office , spot Kendall , an..."
1,22,"((3, 4), Sia)","(10, her)","Get to know Sia at the safe , then greet her p..."
2,23,"((13, 14), Adele)","(20, her)",Could you please place a clothes on the bath t...
3,28,"((1, 2), Martin)","(4, him)","Find Martin , accompany him to the pencle case..."
4,42,"((1, 2), Martin)","(4, him)","Locate Martin , conduct him to the guest room ..."
5,52,"((3, 4), Selena)","(11, her)","Get to know Selena at the shelving unit , then..."
6,63,"((6, 7), Lana)","(13, her)",Could you please get to know Lana at the safe ...
7,68,"((3, 4), Rihanna)","(11, her)","Robot please meet Rihanna at the flower pot , ..."
8,72,"((8, 9), Sia)","(12, her)","Please enter to the bathroom , look for Sia , ..."
9,96,"((7, 8), Rihanna)","(14, her)","Could you pinpoint Kanye , look for Rihanna in..."


In [24]:
#{'_178-it_', '_285-it_', '_226-it_', '_226-it_,', '_95-it_', '_286-it_', '_227-it_', '_179-its_'}
def get_it_s_lab(com_lst):
    pro_tag_set = set()
    for com in com_lst:
        lst = com.split()
        for ele in lst:
            if ele[0] == '_':
                pro_tag_set.add(ele)
    return pro_tag_set



#"referral" : The referring wildcard.
#"referee" : The wildcard being referred.
def store_it_s_coref(com_lst):
    clean_com_lst = com_lst.copy()
    pro_dict = {'idx':[],'referee':[],'referral':[],'words':[]}
    pro_tag_set = get_it_s_lab(com_lst)
    for i,com in enumerate(com_lst):

        # if i == 50:
        #     break

        lst = com.split()

        the_tag = None
        for tag in list(pro_tag_set):
            if tag in lst:
                the_tag = [ele for ele in lst if ele[0] == '_'][0]
                break

        if the_tag != None: # has pronoun in this command
            clean_com_lst[i] = ' '.join([ele for ele in lst if ele[0] != '_'])
            # ------------ store command idx ------------
            pro_dict['idx'].append(i)

            words_lst = [ele for ele in lst if ele[0] != '_' and ele[0] != '-']
            pro_dict['words'].append(' '.join(words_lst))
            # ------------ store referral, the pronoun in word command ------------
            if 'its' in the_tag:
                pro_loc = words_lst.index('its')
                pro_dict['referral'].append((pro_loc,'its'))
            elif 'it' in the_tag:
                pro_loc = words_lst.index('it')
                pro_dict['referral'].append((pro_loc,'it'))

            # ------------ store referee, the word being referred ------------
            raw_loc = lst.index(the_tag)
            search_lst = lst[:raw_loc][::-1]
            obj_tag_count = 0
            i_count = 0
            rev_referee = []
            # print(search_lst)
            for j,ele in enumerate(search_lst):
                if obj_tag_count == 2:
                    rev_referee.append(ele)
                    if search_lst[j] == '-i-':
                        rev_referee.pop()
                        i_count += 1
                        continue

                    elif i_count ==len(rev_referee): # unfinished storing referee
                        continue

                    else:
                        break

                if '_obj-' in ele:
                    obj_tag_count += 1

            referee_lst = rev_referee[::-1]
            referee_loc = find_sub_list(referee_lst,words_lst)
            str_referee = ' '.join(referee_lst).strip()
            pro_dict['referee'].append((referee_loc,str_referee))

    return pro_dict,clean_com_lst

# get_it_s_lab(com_lst)
pro_dict,clean_com_lst = store_it_s_coref(com_lst)
it_df =pd.DataFrame(pro_dict)
it_df

,idx,referee,referral,words
0,112,"((3, 4), file)","(9, it)",Pick up a file from the safe and put it into t...
1,120,"((3, 4), equipments)","(9, it)",Pick up a equipments from the vanity and put i...
2,125,"((3, 4), file)","(10, it)",Pick up a file from the shelving unit and put ...
3,126,"((8, 9), pants)","(12, it)","Navigate to the guest room , spot the pants , ..."
4,150,"((5, 6), products)","(12, it)",Pick up a body care products from the pencle c...
5,180,"((3, 4), shoes)","(9, it)",Pick up a shoes from the closet and put it int...
6,212,"((5, 6), products)","(12, it)",Pick up a body care products from the bath tub...
7,220,"((3, 4), stationary)","(9, it)",Pick up a stationary from the safe and put it ...
8,305,"((3, 4), stationary)","(10, it)",Pick up a stationary from the bath tub and put...
9,313,"((3, 4), clothes)","(9, it)",Pick up a clothes from the safe and put it int...


In [25]:
def merge_df(com_lst,it_df,per_df):
    it_idx = it_df['idx'].tolist()
    per_idx = per_df['idx'].tolist()
    dic = {'idx':[],'referee':[],'referral':[],'words':[]}
    for i,com in enumerate(com_lst):
        if i not in it_idx and i not in per_idx:
            dic['idx'].append(i)
            dic['referee'].append('')
            dic['referral'].append('')
            words_lst = [ele for ele in com.split() if ele[0] != '_' and ele[0] != '-']
            dic['words'].append(' '.join(words_lst))

    rest_df = pd.DataFrame(dic)
    df = it_df.copy().append(per_df)
    df = df.append(rest_df)
    df = df.sort_values('idx')
    return df

pro_df = merge_df(clean_com_lst,it_df,per_df)
pro_df

,idx,referee,referral,words
0,0,,,Say hello to the person wearing earrings in th...
1,1,,,Robot please tell me how many men are leaning ...
2,2,,,Take the person at the shoe rack to the exit
3,3,,,Come behind the child in the bathroom
4,4,,,Could you offer something to drink to all the ...
...,...,...,...,...
910,969,,,Hand me some file from the microwave
911,970,,,Could you take the shoes at the shelving unit'...
912,971,,,"Could you please look for Selena , escort her ..."
913,972,,,Put the skirt into the oven please


In [26]:
def gen_pro_labels(pro_df):
    referee_lst = pro_df['referee'].tolist()
    referral_lst = pro_df['referral'].tolist()
    words_lst = [ele.split() for ele in pro_df['words'].tolist()]
    pro_labels = [['O']*len(lst) for lst in words_lst]
    for sample_idx in range(len(words_lst)):
        if referral_lst[sample_idx] != '':
            referral_idx = referral_lst[sample_idx][0]
            pro_labels[sample_idx][referral_idx] = 'referral'

            referee_start = referee_lst[sample_idx][0][0]
            referee_end = referee_lst[sample_idx][0][1]
            pro_labels[sample_idx][referee_start] = 'B-referee'
            length = referee_end - referee_start
            if length > 1:
                pro_labels[sample_idx][referee_start + 1 : referee_end] = ['I-referee']*(length - 1)

    return pro_labels
pro_labels = gen_pro_labels(pro_df)
print(pro_labels[:10])

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-referee', 'O', 'O', 'O', 'referral'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [27]:
import itertools

def improve_labels(com_lst):
    res = []
    for i,com in enumerate(com_lst):
        # print(i)
        # print(com)
        com = com.split()
        if '-i-' not in com:
            res.append(com)
            continue

        #deal with -i-
        elif '-i-' in com:
            string = ' '.join(com)
            while '-i- -i-' in string:
                string = string.replace('-i- -i-', '-i-')

            while '-i- -' in string:
                string = string.replace('-i- -','-')

            com = string.split()

            idx_lst = []
            for idx,ele in enumerate(com):
                if ele == '-i-':
                    if len(idx_lst) == 0:
                        idx_lst.append([idx])
                    elif idx == idx_lst[-1][-1] + 2:
                        idx_lst[-1].append(idx)
                    else:
                        idx_lst.append([idx])

            if i == 989:
                print(com)
                print(idx_lst)

            for lst in idx_lst:
                end_label = com[lst[-1]+2]
                #com[lst[0]] = end_label.replace(',','')
                if 'B' in end_label:
                    com[lst[-1]+2] = com[lst[-1]+2].replace('B','I')#.replace(',','')

                for j,idx in enumerate(lst):
                    if j == 0:
                        com[idx] = end_label ################################################
                    else:
                        label = com[lst[-1]+2]

                        com[idx] = label

            labelled = com.copy()

        res.append(labelled)
    return res

res = improve_labels(clean_com_lst)
# ' '.join(res[3])
res[0] #989

['Say',
 '-B_greet-',
 'hello',
 '-I_greet-',
 'to',
 'the',
 'person',
 '-B_greet_per-',
 'wearing',
 '-I_greet_per-',
 'earrings',
 '-I_greet_per-',
 'in',
 'the',
 'guest',
 '-B_greet_dest-',
 'room',
 '-I_greet_dest-']

In [28]:
def generate_seq_out(res):
    seq_in = []
    seq_out = []
    for i,com in enumerate(res):
        in_lst = []
        clean_out_lst = []
        for ele in com:
#             print('ele; ',ele)
            if ele[0] != '-':
                in_lst.append(ele)
                clean_out_lst.append('O')
            else:
                clean_out_lst[-1] = ele.replace('-','',2)
                if ',' in ele:
                    clean_out_lst[-1] = clean_out_lst[-1].replace(',','')

        seq_in.append(in_lst)
        seq_out.append(clean_out_lst)

    return seq_in, seq_out

seq_in, seq_out = generate_seq_out(res)
print(seq_in[0])
seq_out[0]

['Say', 'hello', 'to', 'the', 'person', 'wearing', 'earrings', 'in', 'the', 'guest', 'room']


['B_greet',
 'I_greet',
 'O',
 'O',
 'B_greet_per',
 'I_greet_per',
 'I_greet_per',
 'O',
 'O',
 'B_greet_dest',
 'I_greet_dest']

# Check tell commands

In [29]:
for i,(com,lab) in enumerate(zip(seq_in,seq_out)):
    if 'B_tell' in lab:
        print(i)
        print(com)
        print(lab)
        print('-----------------------------')
    if i == 1000:
        break


1
['Robot', 'please', 'tell', 'me', 'how', 'many', 'men', 'are', 'leaning', 'against', 'the', 'wall', 'in', 'the', 'garden']
['O', 'O', 'B_tell', 'B_tell_dest', 'B_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what']
-----------------------------
10
['Please', 'tell', 'me', 'how', 'many', 'stationary', 'items', 'are', 'in', 'the', 'drying', 'rack']
['O', 'B_tell', 'B_tell_dest', 'B_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what']
-----------------------------
11
['Tell', 'me', 'how', 'many', 'gray', 'file', 'objects', 'are', 'in', 'the', 'vanity', 'please']
['B_tell', 'B_tell_dest', 'B_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'I_tell_what', 'O']
-----------------------------
16
['Please', 'tell', 'me', 'how', 'many', 'men'

In [30]:
# need to process the pro df as well
def rem_say(seq_in,seq_out,pro_labels):
    say_idx = [i for i,labels in enumerate(seq_out) if 'B_say' in labels]
    seq_in = [seq for i,seq in enumerate(seq_in) if i not in  say_idx]
    seq_out = [seq for i,seq in enumerate(seq_out) if i not in  say_idx]
    seq_pro = [seq for i,seq in enumerate(pro_labels) if i not in  say_idx]
    return seq_in, seq_out, seq_pro

def rem_dups(com_lst, lab_lst, pro_labels,rem = True):
    com_lst = [' '.join(lst) for lst in com_lst]
    lab_lst = [' '.join(lst) for lst in lab_lst]
    if rem:
        com_lst, lab_lst, seq_pro = rem_say(com_lst,lab_lst,pro_labels)
    else:
        seq_pro = pro_labels

    df = pd.DataFrame (com_lst, columns = ['command'])
    df['label'] = lab_lst
    df['pronouns'] = seq_pro
    df2 = df.drop_duplicates(subset=["command"], keep='first')
    return df2

df = rem_dups(seq_in,seq_out,pro_labels, rem = False)
df

,command,label,pronouns
0,Say hello to the person wearing earrings in th...,B_greet I_greet O O B_greet_per I_greet_per I_...,"[O, O, O, O, O, O, O, O, O, O, O]"
1,Robot please tell me how many men are leaning ...,O O B_tell B_tell_dest B_tell_what I_tell_what...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,Take the person at the shoe rack to the exit,B_guide O B_guide_per O O B_guide_sour I_guide...,"[O, O, O, O, O, O, O, O, O, O]"
3,Come behind the child in the bathroom,B_follow I_follow O B_follow_per O O B_follow_...,"[O, O, O, O, O, O, O]"
4,Could you offer something to drink to all the ...,O O B_take B_take_obj I_take_obj I_take_obj O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...
969,Hand me some file from the microwave,B_take B_take_dest O B_take_obj O O B_take_sour,"[O, O, O, O, O, O, O]"
970,Could you take the shoes at the shelving unit'...,O O B_take O B_take_obj O O B_take_sour I_take...,"[O, O, O, O, O, O, O, O, O, O, O, O, O]"
971,"Could you please look for Selena , escort her ...",O O O B_find I_find B_find_per O B_guide B_gui...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
972,Put the skirt into the oven please,B_take O B_take_obj O O B_take_dest O,"[O, O, O, O, O, O, O]"


In [31]:
def write_unique_intent(df):
    label = df['label'].tolist()
    label_lst = [l.split() for l in label]
    # unique_labels
    intents = [ele[2:]  for lst in label_lst for ele in lst  if ele.count('_')==1] #ele[2:] to remove the B_, I_
    #intents.extend(['PAD', 'O'])
    unique_intent_verb = [*set(intents)]
    unique_intent_B = ['B-' + intent for intent in unique_intent_verb]
    unique_intent_I = ['I-' + intent for intent in unique_intent_verb]

    # Interleave
    unique_intent = unique_intent_B + unique_intent_I
    unique_intent[::2] = unique_intent_B
    unique_intent[1::2] = unique_intent_I

    unique_intent = ['PAD','O'] + unique_intent

    with open('intent_label.txt','w') as f:
        for intent in unique_intent:
            f.write(intent)
            f.write('\n')
    return unique_intent

unique_intent = write_unique_intent(df)
unique_intent

['PAD',
 'O',
 'B-know',
 'I-know',
 'B-greet',
 'I-greet',
 'B-take',
 'I-take',
 'B-guide',
 'I-guide',
 'B-answer',
 'I-answer',
 'B-follow',
 'I-follow',
 'B-go',
 'I-go',
 'B-tell',
 'I-tell',
 'B-find',
 'I-find']

# Train Dev Test split

In [13]:
# from sklearn.model_selection import train_test_split
#
# def split(df):
#     df_train, df_test_dev = train_test_split(df, test_size=0.25, random_state=42)
#     df_dev, df_test = train_test_split(df_test_dev, test_size=0.8, random_state=42)
#
#     env_dict = {'train' : df_train,
#                'dev' : df_dev,
#                'test' : df_test}
#     for df in env_dict.values():
#         df.columns = ['seq_in', 'seq_out', 'seq_pro']
#     return env_dict
#
# env_dict = split(df)
# for env,df in env_dict.items():
#     print(env,'\n')
#     display(df)
#     print('------------------------------------------------------------------')

In [12]:
def find_sub_list(subl, l):
    # s_str = ' '.join(subl)
    # f_str = ' '.join(l)
    # num = f_str.count(s_str)
    # print(num)
    # ind_subl = []
    # for i in range(num):
    #     ind_subl.extend([i for i in range(len(l)) if l[i] in subl])
    ind_subl = [i for i in range(len(l)) if l[i] in subl]
    if len(ind_subl) == 1:
        return [ind_subl[0]]
    else:
        return ind_subl

l='O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what', 'I-what'
find_sub_list(['I-what', 'O', 'I-what'],l)


[0, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
def seq_intent_out(com_lst,lab_lst):
    out_int_lst = []

    #polite = ['Please','Could you', 'Robot please', 'Could you please', 'please']
    polite_unique = ['Please','Could you', 'Robot', 'please']

    #sep = ['and', ',' , ', and']
    sep_unique = ['and', ',' ]
    for idx,(com,lab) in enumerate(zip(com_lst,lab_lst)):

        sentence = ' '.join(com)

        polite_dict = {key:find_sub_list(key.split(' '),com) for key in polite_unique if key in sentence}
        polite_idx = []
        for ele in polite_dict.values():
            polite_idx.extend(ele)

        sep_dict = {key:find_sub_list(key.split(' '),com) for key in sep_unique if key in sentence}
        sep_idx = []
        for ele in sep_dict.values():
            sep_idx.extend(ele)

        non_intent_idx = polite_idx + sep_idx
        lst = ['O' if i in non_intent_idx else None for i in range(len(lab))]

        if 'get to know' in ' '.join(com):
            get_to_idx = find_sub_list(['get','to'],com)
            for j in get_to_idx:
                lst[j] = 'O'

        if 'Get to know' in ' '.join(com):
            get_to_idx = find_sub_list(['Get','to'],com)
            for j in get_to_idx:
                lst[j] = 'O'

        if 'then' in com:
            j = com.index('then')
            lst[j] = 'O'


        intent_lst = []
        B = True
        for i,ele in enumerate(lst):
            if ele == None and B ==True:
                B = False
                if lab[i] != 'O':
                    intent = lab[i].split('_')[1]
                else:
                    intent = intent_lst[-1][0]
                intent_lst.append([intent,[i]])
            elif ele == None and B == False:
                intent_lst[-1][1].append(i)

            elif ele == 'O':
                B = True

        intent_seq = lst#[ele if ele.count('_') != 1 else ele.replace('_','-') for ele in lab]
        for instance_lst in intent_lst:
            # for subtask_lst in instance_lst:
            intent, idx_lst =  instance_lst[0], instance_lst[1]
            begin = idx_lst.pop(0)
            intent_seq[begin] = 'B-' + intent
            for i in idx_lst:
                intent_seq[i] = 'I-' + intent


        out_int_lst.append(intent_seq)

        # if idx == 8:
        if 'I_tell_what' in intent_seq:
            print('  '.join(com))
            print('lab: ', lab)
            print('polite_dict: ',polite_dict)
            print('sep_dict: ',sep_dict)
            print('lst: ', lst)
            print('intent_lst: ',intent_lst)
            print(intent_seq)
            print('-----------------------------------------------------------')
    return out_int_lst


#
lab_lst = [ele.split() for ele in df['label'].tolist()]
com_lst = [ele.split() for ele in df['command'].tolist()]
seq_intent_out(com_lst,lab_lst)

[['B-greet',
  'I-greet',
  'I-greet',
  'I-greet',
  'I-greet',
  'I-greet',
  'I-greet',
  'I-greet',
  'I-greet',
  'I-greet',
  'I-greet'],
 ['O',
  'O',
  'B-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell',
  'I-tell'],
 ['B-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'I-guide',
  'I-guide'],
 ['B-follow',
  'I-follow',
  'I-follow',
  'I-follow',
  'I-follow',
  'I-follow',
  'I-follow'],
 ['O',
  'O',
  'B-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take'],
 ['O',
  'O',
  'O',
  'B-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take',
  'I-take'],
 ['O',
  'O',
  'O',
  'B-follow',
  'I-follow',
  'I-follow',
  'I-follow',
  'I-follow',
  'I-follow'],
 ['B-follow',
  'I-follow',
  'I-follow',
  'I-follow',
  'I-follow',
  

In [14]:
def write(f_name,data_lst, space = True):
    with open(f_name, 'w') as f:
        for i,lst in enumerate(data_lst):
            if space == True:
                # print(i)
                # print(lst)
                com = ' '.join(lst)
            else:
                com = ''.join(lst)
            f.write(com)
            f.write('\n')
    return

def gen_files(env_name, data_df):
    # get seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]

    #get seq in
    com_lst = [ele.split() for ele in data_df['seq_in'].tolist()]
    #get pro labels lst
    pro_labels = data_df['seq_pro'].tolist()

    int_lst = seq_intent_out(com_lst,lab_lst)

    # get label
    label = []
    for lab_seq in int_lst:
        lst = []
        for lab in lab_seq:
            if lab == 'O':
                continue
            elif len(lst) == 0:
                lst.append(lab)
            elif lab not in lst:
                lst.append(lab)
        hash_taged = [ele+'#' if i != len(lst)-1 else ele for i,ele in enumerate(lst)]
        label.append(hash_taged)


    # get seq out
    # 1. remove the intent label out of the seq out
    lab_lst = [ele.split() for ele in data_df['seq_out'].tolist()]
    lab_lst = [[ele if ele.count('_') == 2 else 'O' for ele in lst] for lst in lab_lst]
    # 2. remove the intentions names in the slot names
    slot_seq = []
    for ite,one_seq in enumerate(lab_lst):
        lst = []
        for slot in one_seq:
            if slot == 'O':
                lst.append('O')
            else:
                # with IBO
                slot_start = slot.find('_',2)+1
                entity_type = slot[slot_start:]
                lst.append(slot[0]+'-'+entity_type)


        temp = ' '.join(lst)
        if 'I-what O I-what' in temp:
            lst = temp.replace('I-what O I-what','I-what I-what I-what').split()

        slot_seq.append(lst)

    df = data_df.copy()
    df['label'] = label
    df['seq_intent'] = int_lst
    df['seq_out'] = slot_seq
    df['seq_pro'] = pro_labels

    # write all files
    write(f'./{env_name}/seq.in',com_lst)
    write(f'./{env_name}/seq.out',slot_seq)
    write(f'./{env_name}/seq_intent.out',int_lst)
    write(f'./{env_name}/seq_pro.out',pro_labels)
    write(f'./{env_name}/label',label, False)

    return df

df.columns = ['seq_in', 'seq_out', 'seq_pro']
df_all = gen_files('all',df)
df_all

,seq_in,seq_out,seq_pro,label,seq_intent
0,Say hello to the person wearing earrings in th...,"[O, O, O, O, B-per, I-per, I-per, O, O, B-dest...","[O, O, O, O, O, O, O, O, O, O, O]","[B-greet#, I-greet]","[B-greet, I-greet, I-greet, I-greet, I-greet, ..."
1,Robot please tell me how many men are leaning ...,"[O, O, O, B-dest, B-what, I-what, I-what, I-wh...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-tell#, I-tell]","[O, O, B-tell, I-tell, I-tell, I-tell, I-tell,..."
2,Take the person at the shoe rack to the exit,"[O, O, B-per, O, O, B-sour, I-sour, O, O, B-dest]","[O, O, O, O, O, O, O, O, O, O]","[B-guide#, I-guide]","[B-guide, I-guide, I-guide, I-guide, I-guide, ..."
3,Come behind the child in the bathroom,"[O, O, O, B-per, O, O, B-sour]","[O, O, O, O, O, O, O]","[B-follow#, I-follow]","[B-follow, I-follow, I-follow, I-follow, I-fol..."
4,Could you offer something to drink to all the ...,"[O, O, O, B-obj, I-obj, I-obj, O, B-dest, I-de...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
...,...,...,...,...,...
969,Hand me some file from the microwave,"[O, B-dest, O, B-obj, O, O, B-sour]","[O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."
970,Could you take the shoes at the shelving unit'...,"[O, O, O, O, B-obj, O, O, B-sour, I-sour, I-so...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[B-take#, I-take]","[O, O, B-take, I-take, I-take, I-take, I-take,..."
971,"Could you please look for Selena , escort her ...","[O, O, O, O, O, B-per, O, O, B-per, O, O, B-de...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B-find#, I-find#, B-guide#, I-guide#, B-answe...","[O, O, O, B-find, I-find, I-find, O, B-guide, ..."
972,Put the skirt into the oven please,"[O, O, B-obj, O, O, B-dest, O]","[O, O, O, O, O, O, O]","[B-take#, I-take]","[B-take, I-take, I-take, I-take, I-take, I-tak..."


# Select reasonable Command

In [16]:
del_idx_lst = [1,4,5,6,7,20,27,33,36,49,71,79,85,112]

def write_checked(f_name,data_lst):
    with open(f_name, 'w') as f:
        for s in data_lst:
            f.write(s + '\n')
    return

def write_checked_files(del_idx_lst):
    label = [line.strip() for line in open('all/label')]
    seq_in = [line.strip() for line in open('all/seq.in')]
    seq_out = [line.strip() for line in open('all/seq.out')]
    seq_intent_out = [line.strip() for line in open('all/seq_intent.out')]
    seq_pro_out = [line.strip() for line in open('all/seq_pro.out')]

    checked_label = []
    checked_seq_in = []
    checked_seq_out = []
    checked_seq_intent_out = []
    checked_seq_pro_out = []
    idx = 0
    while len(checked_seq_in) < 100:
        if idx +1 not in del_idx_lst:
            checked_label.append(label[idx])
            checked_seq_in.append(seq_in[idx])
            checked_seq_out.append(seq_out[idx])
            checked_seq_intent_out.append(seq_intent_out[idx])
            checked_seq_pro_out.append(seq_pro_out[idx])
        idx += 1

    write_checked('checked/label',checked_label)
    write_checked('checked/seq.in',checked_seq_in)
    write_checked('checked/seq.out',checked_seq_out)
    write_checked('checked/seq_intent.out',checked_seq_intent_out)
    write_checked('checked/seq_pro.out',checked_seq_pro_out)
    return
write_checked_files(del_idx_lst)

In [17]:
for i,slot in enumerate(df_all['seq_out'].tolist()):
    lab = ['O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what']
    lab1 = ['O', 'O', 'O', 'B-dest', 'B-what', 'I-what', 'I-what', 'O', 'I-what', 'I-what', 'I-what', 'I-what']
    lab2 = ['O', 'O', 'O', 'B-obj', 'O', 'O', 'B-sour', 'O', 'O', 'O', 'B-obj', 'O', 'B-dest', 'O', 'O', 'B-dest', 'O', 'O', 'O', 'B-obj', 'I-obj']
    if ' '.join(slot) == ' '.join(lab) or ' '.join(slot) == ' '.join(lab2) :
        print(df_all['seq_in'].tolist()[i])


In [18]:
def write_unique_slot(df):
    label_lst = df['seq_out'].tolist()
    slots = [ele  for lst in label_lst for ele in lst if ele != 'O'] #ele[2:] to remove the B_, I_
    unique_slot = ['PAD','O'] + [*set(slots)]

    with open('slot_label.txt','w') as f:
        for slot in unique_slot:
            f.write(slot)
            f.write('\n')
    return unique_slot

unique_slot = write_unique_slot(df_all)
unique_slot

['PAD',
 'O',
 'B-what',
 'I-obj',
 'B-obj',
 'B-dest',
 'I-what',
 'I-sour',
 'I-dest',
 'B-sour',
 'B-per',
 'I-per']

In [19]:
def show_unique_labels(env):
    # uni_slot = [line.strip() for line in open('slot_label.txt', 'r', encoding='utf-8')]
    # uni_intent = [line.strip() for line in open('intent_label.txt', 'r', encoding='utf-8')]

    slot_path =  f'./{env}/seq.out'
    intent_path = f'./{env}/seq_intent.out'

    slot_seq = [line.strip().split() for line in open(slot_path, 'r', encoding='utf-8')]
    intent_seq = [line.strip().split() for line in open(intent_path, 'r', encoding='utf-8')]

    slot_lst = [label for seq in slot_seq for label in seq]
    intent_lst = [label for seq in intent_seq for label in seq]

    env_uni_slot = [*set(slot_lst)]
    env_uni_intent = [*set(intent_lst)]
    print('env: ',env)
    print('Unique intent: ',env_uni_intent,len(env_uni_intent))
    print('Unique slots:  ',env_uni_slot,len(env_uni_slot),'\n')
    return


show_unique_labels('all')

env:  all
Unique intent:  ['B-find', 'B-guide', 'B-answer', 'I-know', 'B-take', 'I-tell', 'B-tell', 'B-follow', 'I-follow', 'I-answer', 'I-greet', 'I-go', 'B-greet', 'O', 'B-know', 'I-take', 'I-find', 'B-go', 'I-guide'] 19
Unique slots:   ['B-what', 'I-obj', 'B-obj', 'B-dest', 'O', 'I-what', 'I-dest', 'B-sour', 'I-sour', 'B-per', 'I-per'] 11 

